In [1]:
install.packages('ISLR2')
library(ISLR2)
attach(Wage)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
str(Wage)

'data.frame':	3000 obs. of  11 variables:
 $ year      : int  2006 2004 2003 2003 2005 2008 2009 2008 2006 2004 ...
 $ age       : int  18 24 45 43 50 54 44 30 41 52 ...
 $ maritl    : Factor w/ 5 levels "1. Never Married",..: 1 1 2 2 4 2 2 1 1 2 ...
 $ race      : Factor w/ 4 levels "1. White","2. Black",..: 1 1 1 3 1 1 4 3 2 1 ...
 $ education : Factor w/ 5 levels "1. < HS Grad",..: 1 4 3 4 2 4 3 3 3 2 ...
 $ region    : Factor w/ 9 levels "1. New England",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ jobclass  : Factor w/ 2 levels "1. Industrial",..: 1 2 1 2 2 2 1 2 2 2 ...
 $ health    : Factor w/ 2 levels "1. <=Good","2. >=Very Good": 1 2 1 2 1 2 2 1 2 2 ...
 $ health_ins: Factor w/ 2 levels "1. Yes","2. No": 2 2 1 1 1 1 1 1 1 1 ...
 $ logwage   : num  4.32 4.26 4.88 5.04 4.32 ...
 $ wage      : num  75 70.5 131 154.7 75 ...


In [8]:
install.packages('caret')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




In [9]:
library(tidyverse)
library(caret)
set.seed(123)
training.samples<-Wage$wage %>%
   createDataPartition(p=0.8,list=FALSE)
train.data<-Wage[training.samples, ]
test.data<-Wage[-training.samples, ]

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift




In [11]:
# Modelo de regresión lineal en función de año, edad y educación

modeloRL = lm(formula = wage ~ year + age + education, data = train.data)
summary(modeloRL)
predictions<-modeloRL %>% predict(test.data)
data.frame(
  RMSE=RMSE(predictions,test.data$wage),
  R2=R2(predictions,test.data$wage)
)


Call:
lm(formula = wage ~ year + age + education, data = train.data)

Residuals:
     Min       1Q   Median       3Q      Max 
-107.421  -19.640   -3.631   14.528  219.395 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.303e+03  7.226e+02  -1.803   0.0715 .  
year                         6.801e-01  3.603e-01   1.887   0.0592 .  
age                          5.617e-01  6.368e-02   8.820  < 2e-16 ***
education2. HS Grad          1.129e+01  2.765e+00   4.084 4.56e-05 ***
education3. Some College     2.357e+01  2.904e+00   8.115 7.66e-16 ***
education4. College Grad     3.930e+01  2.888e+00  13.609  < 2e-16 ***
education5. Advanced Degree  6.415e+01  3.124e+00  20.537  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 35.8 on 2395 degrees of freedom
Multiple R-squared:  0.2577,	Adjusted R-squared:  0.2559 
F-statistic: 138.6 on 6 and 2395 DF,  p-value: < 2.2e-16


RMSE,R2
<dbl>,<dbl>
36.28799,0.2771217


In [13]:
install.packages('gam')
library(gam)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: splines

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loaded gam 1.22-5




In [18]:
gam.m3 <- gam(wage ~ s(year , 6) + s(age , 7) + education ,
data = train.data)
summary(gam.m3)


Call: gam(formula = wage ~ s(year, 6) + s(age, 7) + education, data = train.data)
Deviance Residuals:
    Min      1Q  Median      3Q     Max 
-110.67  -19.64   -3.11   14.57  213.07 

(Dispersion Parameter for gaussian family taken to be 1223.35)

    Null Deviance: 4134235 on 2401 degrees of freedom
Residual Deviance: 2916465 on 2384 degrees of freedom
AIC: 23913.17 

Number of Local Scoring Iterations: NA 

Anova for Parametric Effects
             Df  Sum Sq Mean Sq  F value    Pr(>F)    
s(year, 6)    1   11252   11252   9.1977  0.002449 ** 
s(age, 7)     1  153082  153082 125.1337 < 2.2e-16 ***
education     4  839435  209859 171.5444 < 2.2e-16 ***
Residuals  2384 2916465    1223                       
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Anova for Nonparametric Effects
            Npar Df  Npar F  Pr(F)    
(Intercept)                           
s(year, 6)        5  0.7044 0.6202    
s(age, 7)         6 20.1256 <2e-16 ***
education                

In [19]:
predictions<- gam.m3 %>% predict(test.data)
data.frame(
  RMSE=RMSE(predictions,test.data$wage),
  R2=R2(predictions,test.data$wage)
)

RMSE,R2
<dbl>,<dbl>
35.96274,0.2891908
